# ♪民泊サービスにおける物件データを利用した宿泊価格予測モデルの作成

In [1]:
# データ分析・整理
import pandas as pd
import numpy as np
import random as rnd

# 可視化
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 機械学習
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# データ取得
train_df = pd.read_csv('../Accommodation-price-forecast-model/データ/train.csv')
test_df = pd.read_csv('../Accommodation-price-forecast-model/データ/test.csv')
combine = [train_df, test_df]

In [3]:
# データのカラムチェック
print(train_df.columns.values)

In [4]:
# データのプレチェック
train_df.head()

In [5]:
train_df.tail()

In [6]:
# データ情報チェック
train_df.info()
print('_'*40)
test_df.info()

In [7]:
train_df.describe()

In [8]:
train_df.describe(include=['O'])

In [9]:
train_df[['city', 'y']].groupby(['city'], as_index=False).mean().sort_values(by='y', ascending=False)

In [10]:
train_df[["accommodates", "y"]].groupby(['accommodates'], as_index=False).mean().sort_values(by='y', ascending=False)

In [11]:
train_df[["beds", "y"]].groupby(['beds'], as_index=False).mean().sort_values(by='y', ascending=False)

In [12]:
train_df[["review_scores_rating", "y"]].groupby(['review_scores_rating'], as_index=False).mean().sort_values(by='y', ascending=False)

In [13]:
grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train_df, col='y', row='accommodates', size=2.2, aspect=1.6)
grid.map(plt.hist, 'beds', alpha=.5, bins=20)
grid.add_legend();

In [14]:
grid = sns.FacetGrid(train_df, col='Embarked')
grid = sns.FacetGrid(train_df, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

In [15]:
grid = sns.FacetGrid(train_df, col='Embarked', hue='Survived', palette={0: 'k', 1: 'w'})
grid = sns.FacetGrid(train_df, row='Embarked', col='Survived', size=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

In [16]:
## データを書き換える
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['amenities', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'description', 'first_review' ,'host_has_profile_pic' ,'host_identity_verified' ,'host_since' ,'instant_bookable' ,'host_response_rate' ,'last_review' ,'latitude' ,'longitude' ,'name' ,'neighbourhood' ,'number_of_reviews', 'property_type', 'room_type', 'thumbnail_url' ,'zipcode'], axis=1)
test_df = test_df.drop(['amenities', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'description', 'first_review' ,'host_has_profile_pic' ,'host_identity_verified' ,'host_since' ,'instant_bookable' ,'host_response_rate' ,'last_review' ,'latitude' ,'longitude' ,'name' ,'neighbourhood' ,'number_of_reviews', 'property_type', 'room_type', 'thumbnail_url' ,'zipcode'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

Before (55583, 29) (18528, 28) (55583, 29) (18528, 28)


('After', (55583, 8), (18528, 7), (55583, 8), (18528, 7))

In [17]:
## カテゴリ特徴量を変換する
for dataset in combine:
    dataset['city'] = dataset['city'].map( {'SF': 5, 'DC': 4, 'Boston': 3, 'LA': 2, 'NYC': 1, 'Chicago': 0} ).astype(int)

In [18]:
## 欠損値の削除
train_df = train_df.dropna()
test_df['bathrooms'] = test_df.fillna(test_df['bathrooms'].mean())
test_df['bedrooms'] = test_df.fillna(test_df['bedrooms'].mean())
test_df['beds'] = test_df.fillna(test_df['beds'].mean())
test_df['review_scores_rating'] = test_df.fillna(test_df['review_scores_rating'].mean())
combine = [train_df, test_df]

In [19]:
# 整数表記に変更
cols = ['bathrooms', 'bedrooms', 'beds', 'review_scores_rating', 'y']
train_df[cols] = train_df[cols].applymap(np.int64)
cols2 = ['bathrooms', 'bedrooms', 'beds', 'review_scores_rating']
test_df[cols2] = test_df[cols2].applymap(np.int64)
combine = [train_df, test_df]

In [20]:
train_df.head()
train_df.info()
print('_'*40)
test_df.info()

,id,accommodates,bathrooms,bedrooms,beds,city,review_scores_rating,y
0,0,6,2,1,4,2,60,138
1,1,2,1,1,1,4,100,42
2,2,2,2,1,1,1,83,64
3,3,2,1,1,1,5,95,166
4,4,2,1,1,1,1,100,164


In [21]:
train_df['rating_band'] = pd.cut(train_df['review_scores_rating'], 5)
train_df[['rating_band', 'y']].groupby(['rating_band'], as_index=False).mean().sort_values(by='rating_band', ascending=True)

In [22]:
for dataset in combine:    
    dataset.loc[ dataset['review_scores_rating'] <= 36, 'review_scores_rating'] = 0
    dataset.loc[(dataset['review_scores_rating'] > 36) & (dataset['review_scores_rating'] <= 52), 'review_scores_rating'] = 1
    dataset.loc[(dataset['review_scores_rating'] > 52) & (dataset['review_scores_rating'] <= 68), 'review_scores_rating'] = 2
    dataset.loc[(dataset['review_scores_rating'] > 68) & (dataset['review_scores_rating'] <= 84), 'review_scores_rating'] = 3
    dataset.loc[ dataset['review_scores_rating'] > 84, 'review_scores_rating'] = 4

In [23]:
combine = [train_df, test_df]
train_df.head()

,id,accommodates,bathrooms,bedrooms,beds,city,review_scores_rating,y
0,0,6,2,1,4,2,2,138
1,1,2,1,1,1,4,4,42
2,2,2,2,1,1,1,3,64
3,3,2,1,1,1,5,4,166
4,4,2,1,1,1,1,4,164


In [24]:
test_df.head(10)

,id,accommodates,bathrooms,bedrooms,beds,city,review_scores_rating
0,0,6,0,0,0,3,0
1,1,3,1,1,1,2,0
2,2,2,2,2,2,1,0
3,3,4,3,3,3,1,0
4,4,3,4,4,4,2,0
5,5,2,5,5,5,2,0
6,6,6,6,6,6,4,0
7,7,5,7,7,7,2,0
8,8,1,8,8,8,5,0
9,9,6,9,9,9,4,0


## ♪ モデル、予測、解決

- ロジスティック回帰
- KNNまたはk-Nearest Neighbors
- サポートベクターマシン
- ナイーブベイズ分類器
- 決定木
- ランダムフォレスト
- パーセプトロン
- 人工ニューラルネットワーク
- RVMまたは関連性ベクトルマシン

In [25]:
train_df = train_df.drop("id", axis=1)
X_train = train_df.drop("y", axis=1)
Y_train = train_df["y"]
X_test  = test_df.drop("id", axis=1).copy()

print(X_train)
print(Y_train)
print(X_test)

In [26]:
# ロジスティック回帰

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

C:\Users\t_toyofuku\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


5.82

ロジスティック回帰を使用して、特徴の作成と目標の完了のための仮定と決定を検証することができます。これは、決定関数の特徴の係数を計算することで行うことができます。

正の係数は、応答の対数オッズを増加させ(したがって、確率を増加させ)、負の係数は、応答の対数オッズを減少させ(したがって、確率を減少させ)ます。

In [27]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
2,beds,-0.068016
1,bedrooms,-0.072263
3,city,-0.121560
4,review_scores_rating,-0.151190
5,y,-0.169288
0,bathrooms,-0.305859


次に、サポートベクターマシンを使用してモデル化します。これは、分類と回帰分析に使用されるデータを分析する学習アルゴリズムを持つ教師付き学習モデルです。2つのカテゴリ**のうちの1つまたは他のカテゴリ**に属するとマークされた訓練サンプルのセットが与えられると、SVM訓練アルゴリズムは、新しいテストサンプルを1つのカテゴリまたは他のカテゴリに割り当てるモデルを構築し、それを非確率的なバイナリ線形分類器にします。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Support_vector_machine)。

このモデルは、ロジスティクス回帰モデルよりも高い信頼度スコアを生成することに注意してください。

In [28]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

6.4

パターン認識では、k-最近傍アルゴリズム（略してk-NN）は、分類と回帰に使用されるノンパラメトリック手法です。サンプルは、隣人の過半数の投票によって分類され、サンプルは、そのk個の最も近い隣人（kは正の整数で、通常は小さい）の中で最も一般的なクラスに割り当てられます。k = 1の場合、オブジェクトは単にその1つの最も近い隣人のクラスに割り当てられます。参考文献[Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)。

KNNの信頼度スコアは、ロジスティック回帰よりは良いが、SVMよりは悪い。

In [29]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

6.17

機械学習では、ナイーブベイズ分類器は、特徴間の強い（ナイーブな）独立性を仮定したベイズの定理の適用に基づく単純な確率的分類器のファミリである。ナイーブベイズ分類器は、学習問題の変数（特徴量）の数に比例した数のパラメータを線形に必要とする、拡張性の高い分類器である。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)。

モデル生成信頼度スコアは、これまでに評価されたモデルの中で最も低い。

In [30]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

0.44

パーセプトロンは，2値分類器（数値のベクトルで表される入力が，ある特定のクラスに属するか否かを決定できる関数）の教師付き学習のためのアルゴリズムである．これは線形分類器の一種であり，特徴ベクトルと重みを組み合わせた線形予測関数に基づいて予測を行う分類アルゴリズムである．このアルゴリズムは、訓練セットの要素を一度に一つずつ処理するという意味で、オンライン学習を可能にしている。参考文献[ウィキペディア](https://en.wikipedia.org/wiki/Perceptron)。

In [31]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

4.41

In [32]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

C:\Users\t_toyofuku\Anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.21

In [33]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

2.35

このモデルは、特徴（木の枝）を目標値（木の葉）に関する結論にマッピングする予測モデルとして決定木を使用します。これらの木構造では、葉はクラス・ラベルを表し、枝はそれらのクラス・ラベルにつながる特徴の接続を表します。対象変数が連続的な値（典型的には実数）を取ることができる決定木は、回帰木と呼ばれる。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Decision_tree_learning)。

モデル信頼度スコアは、これまでに評価されたモデルの中で最も高い。

In [34]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

12.94

次のモデルであるランダムフォレストは、最もポピュラーなモデルの一つです。ランダムフォレストまたはランダムデシジョンフォレストとは、分類や回帰などのためのアンサンブル学習法であり、学習時に多数の決定木（n_estimators=100）を構築し、個々の木のクラス（分類）または平均予測（回帰）のモードとなるクラスを出力することで動作する。参考文献[Wikipedia](https://en.wikipedia.org/wiki/Random_forest)。

モデルの信頼度スコアは、これまでに評価されたモデルの中で最も高い。このモデルの出力(Y_pred)を結果のコンペ提出物の作成に利用することにしました。

In [35]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

12.94

### モデルの評価

これで、すべてのモデルの評価をランク付けして、問題に最も適したモデルを選択することができます。決定木とランダムフォレストのスコアは同じですが、決定木が学習セットにオーバーフィットする癖を修正するため、ランダムフォレストを使用することにしました。

In [36]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,12.94
8,Decision Tree,12.94
0,Support Vector Machines,6.40
1,KNN,6.17
2,Logistic Regression,5.82
5,Perceptron,4.41
6,Stochastic Gradient Decent,2.35
4,Naive Bayes,0.44
7,Linear SVC,0.21


In [37]:
submission = pd.DataFrame({
        "id": test_df["id"],
        "y": Y_pred
    })
submission.to_csv("submit.csv", index=False)